In [1]:
from dgl.data import *
import numpy as np
import torch
import pandas as pd
import pickle
import argparse
from pathlib import Path
import warnings
import re

## Importing the data

In [4]:
# We choose the desired dataset to import
arg = TreeGridDataset
dataset_name = arg.__name__
dataset = arg()

if len(dataset) == 1:
    dgl_Gs = [dataset[0]]
else:
    dgl_Gs = list(dataset)
feature_map = [np.array(graph.ndata['feat']) for graph in dgl_Gs]
label_map = [np.array(graph.ndata['label']) for graph in dgl_Gs]

info = {}

for idx, G in enumerate(dgl_Gs):    
    # G is a dgl graph
    for j in range(len(label_map[idx])):
        info[str(idx) + '_' + str(j)] = label_map[idx][j]
            
df = pd.DataFrame.from_dict(info, orient='index').reset_index().rename(columns={"index": "name", 0: "label"})

train_df_list = []
val_df_list = []
test_df_list = []
for idx, G in enumerate(dgl_Gs):
    num_labels = list(range(len(G.ndata['label'])))
    train_ind = np.random.choice(num_labels,int(len(num_labels)*0.7),replace=False) ## We randomly pick 70% of the data as part of the training set
    train_pattern = str(idx)+'_(' + '|'.join(map(str, train_ind)) + ')$'
    train = df[df['name'].str.contains(re.compile(train_pattern))]
    train_df_list.append(train)
    num_labels = [x for x in num_labels if x not in train_ind]
    
    val_ind = np.random.choice(num_labels,int(len(num_labels)*0.33),replace=False) ## We randomly pick 10% of the data as part of the validation set
    val_pattern = str(idx)+'_(' + '|'.join(map(str, val_ind)) + ')$'
    val = df[df['name'].str.contains(re.compile(val_pattern))]
    val_df_list.append(val)
    num_labels = [x for x in num_labels if x not in val_ind]
    
    test_ind = num_labels
    test_pattern = str(idx)+'_(' + '|'.join(map(str, test_ind)) + ')$'  ## We randomly pick 20% of the data as part of the test set
    test = df[df['name'].str.contains(re.compile(test_pattern))]
    test_df_list.append(test)
    
train_df = pd.concat(train_df_list)
val_df = pd.concat(val_df_list)
test_df = pd.concat(test_df_list)

directory = Path('data/' + str(dataset_name))
if not directory.exists():
    directory.mkdir(parents=True, exist_ok=True)

train_df.reset_index(drop = True).to_csv('data/' + str(dataset_name) + '/train.csv')
val_df.reset_index(drop = True).to_csv('data/' + str(dataset_name) + '/val.csv')
test_df.reset_index(drop = True).to_csv('data/' + str(dataset_name) + '/test.csv')

with open('data/' + str(dataset_name) + '/graph_dgl.pkl', 'wb') as f:
    pickle.dump(dgl_Gs, f)
    
with open('data/' + str(dataset_name) + '/label.pkl', 'wb') as f:
    pickle.dump(info, f)
    
np.save('data/' + str(dataset_name) + '/features.npy', np.array(feature_map))

Done loading data from cached files.


/tmp/ipykernel_220031/96704680.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  train = df[df['name'].str.contains(re.compile(train_pattern))]
/tmp/ipykernel_220031/96704680.py:38: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  val = df[df['name'].str.contains(re.compile(val_pattern))]
/tmp/ipykernel_220031/96704680.py:44: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  test = df[df['name'].str.contains(re.compile(test_pattern))]


## Running the HGRAM

In [57]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --k_qry 5

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=5, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=64, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=1000, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 64x64 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x64 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_manifold): 

In [5]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --k_qry 10

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=10, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=64, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=1000, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 64x64 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x64 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_manifold):

In [6]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --k_qry 20

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=20, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=64, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=1000, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 64x64 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x64 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_manifold):

In [7]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --k_qry 30

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=30, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=64, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=1000, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 64x64 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x64 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_manifold):

In [8]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --batchsz 100

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=24, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=64, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=100, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 64x64 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x64 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_manifold): 

In [9]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --batchsz 500

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=24, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=64, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=500, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 64x64 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x64 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_manifold): 

In [15]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --batchsz 1000

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=24, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=64, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=1000, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 64x64 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x64 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_manifold):

In [16]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --batchsz 2000

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=24, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=64, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=2000, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 64x64 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x64 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_manifold):

In [17]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --hidden_dim 32

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=24, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=32, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=1000, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x32 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 32 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 32x32 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 32 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x32 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_manifold):

In [18]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --hidden_dim 64

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=24, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=64, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=1000, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 64x64 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 64 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x64 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_manifold):

In [19]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --hidden_dim 158

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=24, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=158, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=1000, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x158 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 158 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 158x158 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 158 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x158 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_man

In [20]:
!python train.py --data_dir data/TreeGridDataset/ --task_setup Shared --epoch 3 --train_result_report_steps 100 --hidden_dim 256

Namespace(epoch=3, n_way=3, k_spt=3, k_qry=24, task_num=8, meta_lr=0.001, update_lr=0.001, update_step=5, update_step_test=10, input_dim=1, hidden_dim=256, attention_size=32, data_dir='data/TreeGridDataset/', no_finetune=True, task_setup='Shared', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=100, num_workers=0, batchsz=1000, link_pred_mode='False', h=2, sample_nodes=1000)
There are 2 classes 
Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x256 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 256 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 256x256 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 256 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 2x256 (cuda:0)]
        (5): Parameter containing: [torch.float32 of size 2 (cuda:0)]
    )
    (_manifold): PoincareBall manifold
  )
  (_man